## RNNLM 개선 작업

1) LSTM 계층을 깊게 쌓기
- 연구자가 처한 텍스트 문제의 성격과 데이터 양에 따라서 LSTM 층을 깊게 쌓는 것이 정확도 향상에 도움을 줄 수 있음
- 층을 깊게 쌓는 것은 **모델의 표현력**을 향상 시키는 것이라고도 할 수 있음
- 층의 개수를 설정하는 것 또한 하이퍼 파라미터에 해당됨 -> 적절히 설정하는 것이 하나의 과업

2) Dropout
- 사용자가 설정한 뉴런의 비율만큼 무작위로 무시하고 학습을 진행하는 **정규화 방법**
- 정규화는 모델의 일반화 능력을 향상시키고 과적합 문제를 회피하기 위함임
- 일반적인 Dropuout :
    - RNN, LSTM에서 Dropout을 구현하기 위해서는 시간방향에 Dropuout Layer를 적용하지 않고 동일한 시간t 내의 상, 하위 계층 사이에 Dropout 계층을 쌓는방식을 이용
- 변형 Dropout :
    - RNN, LSTM에서 시간방향 및 계층 방향 모두에 Dropout Layer를 적용한느 방법. 이를 구현하기 위해서 MASK를 도입했는데 이는 동일한 계층에 있는 Dropuout Layer, 시간방향에 진행되는 Dropuout Layer별로 동일한 뉴런들을 MASK하여 나머지 층은 그대로 학습이 진행되도록 하는 방법
    
3) Weight Tying (가중치 공유)
- Embedding 계층의 가중치 **V * H** 와 Affine 꼐층의 가중치 **H * V**를 하나의 가중치로 동일하게 공유하도록 하는 것

In [1]:
# 2층 LSTM과 Dropout, Weight Tying 기법을 적용한 BetterRnnlm 구현
import numpy as np
from mh_common.mh_time_layers import *

import sys
sys.path.append(r'C:\Users\myunghoon_k\OneDrive - UOS\bitamin\dl_nlp_study\deep-learning-from-scratch-2-master')
from common.base_model import BaseModel

class BetterRnnlm(BaseModel):
    
    def __init__(self, vocab_size = 10000, wordvec_size = 650,
                hidden_size = 650, dropout_ratio = 0.5):
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn
        
        embed_W = (rn(V, D) / 100).astype('f')
        lstm_Wx1 = (rn(D, 4*H) / np.sqrt(D)).astype('f')
        lstm_Wh1 = (rn(H, 4*H) / np.sqrt(H)).astype('f')
        lstm_b1 = np.zeros(4*H).astype('f')
        lstm_Wx2 = (rn(D, 4*H) / np.sqrt(D)).astype('f')
        lstm_Wh2 = (rn(H, 4*H) / np.sqrt(H)).astype('f')
        lstm_b2 = np.zeros(4*H).astype('f')
        affine_b = np.zeros(V).astype('f')
        
        self.layers = [
            TimeEmbedding(embed_W),
            TimeDropout(dropout_ratio),
            TimeLSTM(lstm_Wx1, lstm_Wh1, lstm_b1, stateful = True),
            TimeDropout(dropout_ratio),
            TimeLSTM(lstm_Wx2, lstm_Wh2, lstm_b2, stateful = True),
            TimeDropout(dropout_ratio),
            TimeAffine(embed_W.T, affine_b)
        ]
        
        self.loss_layer = TimeSoftmaxWithLoss()
        self.lstm_layers = [self.layers[2], self.layers[4]]
        self.drop_layers = [self.layers[1], self.layers[3], self.layers[5]]
        
        self.params, self.grads = [], []
        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads
            
    def predict(self, xs, train_flg = True):
        for layer in self.drop_layers:
            layer.train_flg = train_flg
            
        for layer in self.layers:
            xs = layer.forward(xs)
        return xs
    
    def forward(self, xs, ts, train_flg = True):
        score = self.predict(xs, train_flg)
        loss = self.loss_layer.forward(score, ts)
        return loss
    
    def backward(self, dout = 1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout
    
    def reset_state(self):
        for layer in self.lstm_layers:
            layer.reset_state()

In [2]:
#모델 학습과정 구현
#Pytorch 에서 제공하는 기능을 추가하여 구현하기
#Epoch마다 Validation 과정에서 perplexity가 낮아질 경우 lr을 이전보다 낮추는 기능을 추가하기
from common import config

config.GPU = True

from mh_common.mh_optimizer import SGD
from common.trainer import RnnlmTrainer
from common.util import eval_perplexity, to_gpu
from dataset import ptb

#하이퍼 파라미터 설정
batch_size = 20
wordvec_size = 650
hidden_size = 650
time_size = 35
lr = 20.0
max_epoch = 40
max_grad = 0.25
dropout = 0.5

#데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
corpus_val, _, _ = ptb.load_data('val')
corpus_test, _, _ = ptb.load_data('test')

if config.GPU:
    corpus = to_gpu(corpus)
    corpus_val = to_gpu(corpus_val)
    corpus_test = to_gpu(corpus_test)
    
vocab_size = len(word_to_id)
xs = corpus[:-1]
ts = corpus[1:]

model = BetterRnnlm(vocab_size, wordvec_size, hidden_size, dropout)
optimizer = SGD(lr)
trainer = RnnlmTrainer(model, optimizer)

best_ppl = float('inf')
for epoch in range(max_epoch):
    trainer.fit(xs, ts, max_epoch = 1, batch_size = batch_size,
               time_size = time_size, max_grad = max_grad)
    
    model.reset_state()
    ppl = eval_perplexity(model, corpus_val)
    print('검증 퍼플렉서티: ', ppl)
    
    if best_ppl > ppl:
        best_ppl = ppl
        model.save_params()
    else:
        lr /= 4.0
        optimizer.lr = lr
        
    model.reset_state()
    print('-' * 50)
    
#테스트 데이터로 평가
model.reset_state()
ppl_test = eval_perplexity(mdoel, corpus_test)
print('테스트 퍼플렉서티: ', ppl_test)
#어쩐 일인지 GPU로 학습이 되지 않음, Cupy는 정상작동함

| 에폭 1 |  반복 1 / 1327 | 시간 1[s] | 퍼플렉서티 10000.32
| 에폭 1 |  반복 21 / 1327 | 시간 39[s] | 퍼플렉서티 2386.54


KeyboardInterrupt: 

In [4]:
xs.shape

(929588,)

In [ ]:
type(corpus)

## RNN을 이용한 문장 생성

- 기존의 RNN모델은 학습을 한 뒤 Softmax 계층에서 확률표현을 하여 input다음에 등장할 단어를 맞추는 작업을 수행
- 이를 바탕으로 input문장이 주어지면 output 문장을 생성하는 모델을 개발하는 작업을 수행하기

In [ ]:
from mh_common.mh_functions import softmax
from ch06.rnnlm import Rnnlm
from ch06.better_rnnlm import BetterRnnlm

class RnnlmGen(Rnnlm):
    def generate(self, start_id, skip_ids = None, sample_size = 100):
        word_ids = [start_id]
        
        x = start_id
        
        while len(word_ids) < sample_size:
            x = np.array(x).reshape(1, 1)
            score = self.predict(x) #RNN층을 통해 학습된 분산표현이 표현된 어떤 값들
            p = softmax(score.flatten()) #분산표현이 softmax층을 통해 확률표현으로 정규화됨
            
            sampled = np.random.choice(len(p), size = 1, p = p)
            if (skip_ids is None) or (sampled not in skip_ids):
                x = sampled
                word_ids.append(int(x))
                
        return word_ids
    
    def get_state(self):
        return self.lstm_layer.h, self.lstm_layer.c
    
    def set_state(self, state):
        self.lstm_layer.set_state(*state)

In [ ]:
#위의 모델로 문장 생성하기
import sys
sys.path.append('..')
from dataset import ptb

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)

model = RnnlmGen()
model.load_params(r"C:\Users\myunghoon_k\OneDrive - UOS\bitamin\dl_nlp_study\deep-learning-from-scratch-2-master\ch06\Rnnlm.pkl")

# start 문자와 skip 문자 설정
start_word = 'you' #첫문자를 you로 설정하여 이 뒤에 올 단어들을 예측
start_id = word_to_id[start_word]
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]

# 문장 생성
word_ids = model.generate(start_id, skip_ids)
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '.\n')
print(txt)

## seq2seq 모델

- sequence to sequence, 즉 시계열 데이터를 받아서 비슷한 형태의 시계열 데이터를 생산하는 모델
- Encoder, Decoder를 모델의 주요구조로 가지며 이 Encoder, Decoder는 각각 RNN형태의 모델로 구성된다
- Encoder
    - input을 받아서 Embedding -> LSTM층으로 보내는 작업을 수행
    - 이전의 Rnnlm, BetterRnnlm은 다음시점의 계층 및 상위 계층으로 h를 보냈으나
    - Encoder내에서는 **상위 계층이 존재하지 않으므로** 다음 t 시점으로만 h를 보냄
    - 또한 LSTM구조상 cell state output이 생성되게 되는데 이는 오직 Encoder내부 학습에서만 사용되며 Decoder로는 전달되지 않는다
    - 따라서 최종 Encoder input의 h만이 **Decoder의 추가적인 input**으로 사용되게 됨
    
### h를 Decoder를 보내는 것은 input의 정보를 응축하여 보내는것으로 이해할 수 있음
### 즉 가변길이의 input정보들을 h라는 고정길이 벡터로 변환하여 보내는 것
    
- Decoder
    - 정답에 해당하는 시계열 데이터를 input으로, Encoder의 최종 h를 또 다른 input으로 사용하여 번역될 단어를 예측
    - 모델 구조는 Rnnlm, BetterRnnlm과 비슷한데, Embedding -> LSTM(여러개 존재 가능) -> Affine -> Softmax 구조를 취함

In [ ]:
#seq2seq의 Toy Problem
#문자열 형태로 된 수식을 학습하여 결과를 내는 모델 만들기
#즉 수식에 대한 사전 규칙 부여 없이 딥러닝 모델로 스스로 수식의 규칙을 학습하도록 하는 것

from dataset import sequence

(x_train, t_train), (x_test, t_test) = sequence.load_data('addition.txt', seed = 1984)

char_to_id, id_to_char = sequence.get_vocab()

print(x_train.shape, t_train.shape)
print(x_test.shape, t_test.shape)

print(x_train[0]) #숫자 문자열의 id가 담긴형태로 데이터가 구성되어 있음
print(x_test[0])

print(''.join([id_to_char[c] for c in x_train[0]]))
print(''.join([id_to_char[c] for c in t_train[0]]))